In [85]:
import pandas as pd
import numpy as np
import datetime
from sas7bdat import SAS7BDAT

In [86]:
rockin = pd.ExcelFile('Rockingham 2014-2018 Homeowners Data-Verisk_copy.xlsx')
# print(rockin.sheet_names) and load each tab into a df
policy = rockin.parse('Policy_Data')
EP = rockin.parse('EP')
Loss = rockin.parse('Losses')

In [90]:
# print(EP.SumOfEARNED.sum()) 76,704,662
# print(Loss.INCURRED.sum())  37,388,158

EP['EFF_YR'] = EP['A_POL_EFF_DATE'].dt.year
EP2 = EP.groupby(['A_POLICY_NO','EFF_YR']).SumOfEARNED.sum().reset_index()
# print(EP2.EFF_YR.count()) 127187

Loss['EFF_YR'] = Loss['POL_EFF_DATE'].dt.year
# Loss = Loss.drop(columns=['EFF_YR']) # OCCUR_NUMBER, POL_EFF_DATE, CAUSE
Loss2 = Loss.groupby(['POLICY_NUMBER', 'EFF_YR']).INCURRED.sum().reset_index()
# print(Loss2.EFF_YR.count()) 4009
# print(Loss2.head())
# print(Loss2.INCURRED.sum()) 37388157.78

EPANDLOSS = EP2.merge(Loss2, left_on=['A_POLICY_NO', 'EFF_YR'], right_on=['POLICY_NUMBER', 'EFF_YR'], how='left')
# print(EPANDLOSS.EFF_YR.count()) 127187
# print(EPANDLOSS.INCURRED.sum()) 37308662.44

policy = policy[(policy.FORM == 3)]
policy = policy.drop_duplicates(subset=['POLICY_NUMBER'])
# print(policy.STATE.count()) 25283
# print(policy.head())

ALLTHREE = EPANDLOSS.merge(policy, left_on=['A_POLICY_NO',], right_on=['POLICY_NUMBER'], how='inner')
ALLTHREE = ALLTHREE[['A_POLICY_NO', 'EFF_YR', 'SumOfEARNED', 'INCURRED', 'STATE', 'ZIPCODE', 
                     'YEAR_BUILT', 'COV_A_AMOUNT', 'DEDUCTIBLE']]
# print(ALLTHREE.A_POLICY_NO.count()) 105777
# print(ALLTHREE.INCURRED.sum()) 30523804.0

LOSSBYYR = ALLTHREE.groupby(['EFF_YR']).INCURRED.sum()
PREMBYYR = ALLTHREE.groupby(['EFF_YR']).SumOfEARNED.sum()
# print(LOSSBYYR)
# print(PREMBYYR)

ALLTHREE.to_csv('Rockingham_MERGED.csv')


In [ ]:
# read in score table and format for lift calculation
with SAS7BDAT('') as file:
    scoretb = 
#